<a href="https://colab.research.google.com/github/JarekMaleszyk/data-science-project-sandbox/blob/main/dual_pos_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Etap 1: Dane z SA

Skrypt należy wykonać na SA, wyeksportować do CSV  i umieścić bezpośrednio w katalogu ./data
```SQL
/*Part 1 - group by*/
proc sql outobs=max;
	create table DICT_MODEL_PRODUCT_CONC_TMP as
    select
        MODEL_PROD_CD,
        catx(', ', MAT_ID) as MAT_IDS
    from CMDICT.DICT_MODEL_PRODUCT
    group by MODEL_PROD_CD;
run;
/*Part 1 - sort*/
proc sort data=WORK.DICT_MODEL_PRODUCT_CONC_TMP;
    by MODEL_PROD_CD;
run;
/*Part 1 - concatenation*/
data DICT_MODEL_PRODUCT_CONC;
    set WORK.DICT_MODEL_PRODUCT_CONC_TMP;
    by MODEL_PROD_CD;
    retain CONCATENATED_MAT_IDS;

    if first.MODEL_PROD_CD
		then CONCATENATED_MAT_IDS = MAT_IDS;
    else CONCATENATED_MAT_IDS = catx('#', CONCATENATED_MAT_IDS, MAT_IDS);

    if last.MODEL_PROD_CD
		then output;
    keep MODEL_PROD_CD CONCATENATED_MAT_IDS;
run;

/*Part 2 - join + union */
proc sql outobs=max;
create table DICT_MODEL_RULES_TST as
	select
		dc.RULESET,
		dc.RULE_ID,
		dc.MODEL_PROD_CD_ITEM1,
		dc.MODEL_PROD_CD_ITEM2,
		dc.MODEL_PROD_CD_ITEM3,
		dc.MODEL_PROD_CD_ITEM4,
		dc.MODEL_PROD_CD_ITEM5,
		dc.RECOMENDATION,
		dc.PRIORITY,
		dc.LIFT,
		dc.SUPPORT,
		dc.COUNT,
		dc.PRIORITY_COUNT,
		dc.PRIORITY_LIFT,
		dc.DZIEN_TYGODNIA,
		dc.PORA_DNIA_OD,
		dc.PORA_DNIA_DO,
		dc.TYP_STACJI,
		dc.SEZON,
		it1.CONCATENATED_MAT_IDS AS MODEL_PROD_LIST_ITEM1,
		it2.CONCATENATED_MAT_IDS AS MODEL_PROD_LIST_ITEM2,
		it3.CONCATENATED_MAT_IDS AS MODEL_PROD_LIST_ITEM3,
		it4.CONCATENATED_MAT_IDS AS MODEL_PROD_LIST_ITEM4,
		it5.CONCATENATED_MAT_IDS AS MODEL_PROD_LIST_ITEM5
	from CMDICT.DICT_MODEL_RULES as dc
	left join WORK.DICT_MODEL_PRODUCT_CONC as it1 on dc.MODEL_PROD_CD_ITEM1 = it1.MODEL_PROD_CD
	left join WORK.DICT_MODEL_PRODUCT_CONC as it2 on dc.MODEL_PROD_CD_ITEM2 = it2.MODEL_PROD_CD
	left join WORK.DICT_MODEL_PRODUCT_CONC as it3 on dc.MODEL_PROD_CD_ITEM3 = it3.MODEL_PROD_CD
	left join WORK.DICT_MODEL_PRODUCT_CONC as it4 on dc.MODEL_PROD_CD_ITEM4 = it4.MODEL_PROD_CD
	left join WORK.DICT_MODEL_PRODUCT_CONC as it5 on dc.MODEL_PROD_CD_ITEM5 = it5.MODEL_PROD_CD
	where 1=1
		and dc.RULESET = "ALL_UNNAMED_DUAL_POS"
		and dc.DZIEN_TYGODNIA = 5
		and dc.TYP_STACJI = "Wiejska"
		and dc.PORA_DNIA_OD = 14
		and dc.PORA_DNIA_DO = 21
		and dc.PRIORITY_COUNT = 1

	UNION ALL

	select
		dc.RULESET,
		dc.RULE_ID,
		dc.MODEL_PROD_CD_ITEM1,
		dc.MODEL_PROD_CD_ITEM2,
		dc.MODEL_PROD_CD_ITEM3,
		dc.MODEL_PROD_CD_ITEM4,
		dc.MODEL_PROD_CD_ITEM5,
		dc.RECOMENDATION,
		dc.PRIORITY,
		dc.LIFT,
		dc.SUPPORT,
		dc.COUNT,
		dc.PRIORITY_COUNT,
		dc.PRIORITY_LIFT,
		dc.DZIEN_TYGODNIA,
		dc.PORA_DNIA_OD,
		dc.PORA_DNIA_DO,
		dc.TYP_STACJI,
		dc.SEZON,
		it1.CONCATENATED_MAT_IDS AS MODEL_PROD_LIST_ITEM1,
		it2.CONCATENATED_MAT_IDS AS MODEL_PROD_LIST_ITEM2,
		it3.CONCATENATED_MAT_IDS AS MODEL_PROD_LIST_ITEM3,
		it4.CONCATENATED_MAT_IDS AS MODEL_PROD_LIST_ITEM4,
		it5.CONCATENATED_MAT_IDS AS MODEL_PROD_LIST_ITEM5
	from CMDICT.DICT_MODEL_RULES as dc
	left join WORK.DICT_MODEL_PRODUCT_CONC as it1 on dc.MODEL_PROD_CD_ITEM1 = it1.MODEL_PROD_CD
	left join WORK.DICT_MODEL_PRODUCT_CONC as it2 on dc.MODEL_PROD_CD_ITEM2 = it2.MODEL_PROD_CD
	left join WORK.DICT_MODEL_PRODUCT_CONC as it3 on dc.MODEL_PROD_CD_ITEM3 = it3.MODEL_PROD_CD
	left join WORK.DICT_MODEL_PRODUCT_CONC as it4 on dc.MODEL_PROD_CD_ITEM4 = it4.MODEL_PROD_CD
	left join WORK.DICT_MODEL_PRODUCT_CONC as it5 on dc.MODEL_PROD_CD_ITEM5 = it5.MODEL_PROD_CD
	where 1=1
		and dc.RULESET = "ALL_VITAY_DUAL_POS"
		and dc.TYP_STACJI = "Wiejska"
		and dc.SEZON = "Z"
		and dc.GR2_TRN_DIST_GASTRO_ALL_CNT_MAX6 = 1
		and dc.PORA_DNIA_OD = 14
		and dc.PORA_DNIA_DO = 21
		and dc.PRIORITY_COUNT = 1
;
run;
```

#### Etap 2: Normalizacja pliku

In [ ]:
import pandas as pd
import numpy as np
import requests
import random

In [ ]:
CSV_FILE_PATH = "data/DICT_MODEL_RULES_TST.csv"

In [ ]:
dtypes = {"MODEL_PROD_CD_ITEM1": "str",
          "MODEL_PROD_CD_ITEM2": "str",
          "MODEL_PROD_CD_ITEM3": "str",
          "MODEL_PROD_CD_ITEM4": "str",
          "MODEL_PROD_CD_ITEM5": "str",
          "MODEL_PROD_LIST_ITEM1": "str",
          "MODEL_PROD_LIST_ITEM2": "str",
          "MODEL_PROD_LIST_ITEM3": "str",
          "MODEL_PROD_LIST_ITEM4": "str",
          "MODEL_PROD_LIST_ITEM5": "str"
         }

In [ ]:
df = pd.read_csv(CSV_FILE_PATH, sep=";", header=0, dtype=dtypes)

In [ ]:
df['MODEL_PROD_LIST_ITEM1'] = df['MODEL_PROD_LIST_ITEM1'].str.split('#')
df['MODEL_PROD_LIST_ITEM2'] = df['MODEL_PROD_LIST_ITEM2'].str.split('#')
df['MODEL_PROD_LIST_ITEM3'] = df['MODEL_PROD_LIST_ITEM3'].str.split('#')
df['MODEL_PROD_LIST_ITEM4'] = df['MODEL_PROD_LIST_ITEM4'].str.split('#')
df['MODEL_PROD_LIST_ITEM5'] = df['MODEL_PROD_LIST_ITEM5'].str.split('#')

In [ ]:
df.sample(n=20)

,RULESET,RULE_ID,MODEL_PROD_CD_ITEM1,MODEL_PROD_CD_ITEM2,MODEL_PROD_CD_ITEM3,MODEL_PROD_CD_ITEM4,MODEL_PROD_CD_ITEM5,RECOMENDATION,PRIORITY,LIFT,...,DZIEN_TYGODNIA,PORA_DNIA_OD,PORA_DNIA_DO,TYP_STACJI,SEZON,MODEL_PROD_LIST_ITEM1,MODEL_PROD_LIST_ITEM2,MODEL_PROD_LIST_ITEM3,MODEL_PROD_LIST_ITEM4,MODEL_PROD_LIST_ITEM5
229,ALL_UNNAMED_DUAL_POS,5271145,ENERGETYKI,NAPOJE GAZOWANE,NaN,NaN,NaN,WODA,0,2.1583,...,5.0,14,21,Wiejska,NaN,"[D05-018136, D05-018139, D05-018138, D05-01824...","[D05-015030, D05-013579, D05-018050, D05-01367...",NaN,NaN,NaN
420,ALL_VITAY_DUAL_POS,2631802,ENERGETYKI,HOT-DOG MEGA,NaN,NaN,NaN,NAPOJE GAZOWANE,0,3.5097,...,NaN,14,21,Wiejska,Z,"[D05-018136, D05-018139, D05-018138, D05-01824...","[D10-000779, D10-000561, D05-008787, D05-00890...",NaN,NaN,NaN
170,ALL_UNNAMED_DUAL_POS,7861704,OLEJ NAPEDOWY,SLODKA PRZEKASKA,NaN,NaN,NaN,SOKI - MARKA WLASNA,0,94.6409,...,5.0,14,21,Wiejska,NaN,"[D01-000040, D01-000055, D01-000039, D01-00001...","[D10-000840, D10-000841, D05-008501, D10-00082...",NaN,NaN,NaN
261,ALL_UNNAMED_DUAL_POS,6696201,HOT-DOG MEGA,OLEJ NAPEDOWY PREMIUM,NaN,NaN,NaN,NAPOJE GAZOWANE,0,2.8668,...,5.0,14,21,Wiejska,NaN,"[D10-000779, D10-000561, D05-008787, D05-00890...",[D01-000030],NaN,NaN,NaN
3,ALL_UNNAMED_DUAL_POS,7693403,OLEJ NAPEDOWY,WODA,NaN,NaN,NaN,NAPOJE GAZOWANE,0,1.4085,...,5.0,14,21,Wiejska,NaN,"[D01-000040, D01-000055, D01-000039, D01-00001...","[D05-018114, D05-001524, D05-018039, D05-01803...",NaN,NaN,NaN
77,ALL_UNNAMED_DUAL_POS,6841465,KANAPKA,WYROBY TYTONIOWE,NaN,NaN,NaN,NAPOJE GAZOWANE,0,4.3778,...,5.0,14,21,Wiejska,NaN,"[D05-018143, D05-018145, D10-000842, D10-00079...","[D06-011079, D06-011078, D06-011077, D06-01107...",NaN,NaN,NaN
234,ALL_UNNAMED_DUAL_POS,3440182,HERBATA,NaN,NaN,NaN,NaN,CAFE LATTE,0,3.2188,...,5.0,14,21,Wiejska,NaN,"[D10-000555, D10-000554, D10-000443, D10-00044...",NaN,NaN,NaN,NaN
198,ALL_UNNAMED_DUAL_POS,7177164,SLODKA PRZEKASKA,SOKI - MARKA WLASNA,NaN,NaN,NaN,KAWA Z MLEKIEM,0,1.1730,...,5.0,14,21,Wiejska,NaN,"[D10-000840, D10-000841, D05-008501, D10-00082...","[D05-017927, D05-017926, D05-017924, D05-01792...",NaN,NaN,NaN
309,ALL_UNNAMED_DUAL_POS,3949763,CHEMIA SAMOCHODOWA,WYROBY TYTONIOWE,NaN,NaN,NaN,NAPOJE GAZOWANE,0,1.8470,...,5.0,14,21,Wiejska,NaN,"[D04-003979, D04-004190, D04-004151, D04-00413...","[D06-011079, D06-011078, D06-011077, D06-01107...",NaN,NaN,NaN
323,ALL_VITAY_DUAL_POS,3032670,HERBATA,NaN,NaN,NaN,NaN,KAWA Z MLEKIEM,0,2.5246,...,NaN,14,21,Wiejska,Z,"[D10-000555, D10-000554, D10-000443, D10-00044...",NaN,NaN,NaN,NaN


In [ ]:
def fix_list(input_list: list, ln: int) -> list:
    '''
    Funkcja zostawia w liście wejściowej tylko te elementy których długość jest równa ln
    '''
    return [x for x in input_list if len(x) == ln]

In [ ]:
df['MODEL_PROD_LIST_ITEM1'] = df['MODEL_PROD_LIST_ITEM1'][df['MODEL_PROD_LIST_ITEM1'].str.len() > 0].apply(lambda MAT_IDS: fix_list(MAT_IDS, 10))
df['MODEL_PROD_LIST_ITEM2'] = df['MODEL_PROD_LIST_ITEM2'][df['MODEL_PROD_LIST_ITEM2'].str.len() > 0].apply(lambda MAT_IDS: fix_list(MAT_IDS, 10))
df['MODEL_PROD_LIST_ITEM3'] = df['MODEL_PROD_LIST_ITEM3'][df['MODEL_PROD_LIST_ITEM3'].str.len() > 0].apply(lambda MAT_IDS: fix_list(MAT_IDS, 10))
df['MODEL_PROD_LIST_ITEM4'] = df['MODEL_PROD_LIST_ITEM4'][df['MODEL_PROD_LIST_ITEM4'].str.len() > 0].apply(lambda MAT_IDS: fix_list(MAT_IDS, 10))
df['MODEL_PROD_LIST_ITEM5'] = df['MODEL_PROD_LIST_ITEM5'][df['MODEL_PROD_LIST_ITEM5'].str.len() > 0].apply(lambda MAT_IDS: fix_list(MAT_IDS, 10))

#### Etap 3: Budowanie pliku json i testowanie API.

In [ ]:
import json
import os
import random
import time
from utils import print_progress_bar
import shutil
import logging

import warnings
warnings.filterwarnings('ignore')

In [ ]:
JSON_TEMPLATE_FILE = "data/dual-pos-request-template.txt"

with open(JSON_TEMPLATE_FILE, 'r') as file:
    template_file = file.read()

In [ ]:
def replace_placeholder_in_text(text: str, placeholder: str, replacement: list[str]) -> str:
    result_replacement = ""
    for item in replacement:
        result_replacement = result_replacement + '["' + item + '"],\t'
    return text.replace(placeholder, result_replacement[:-2] + "\t")

In [ ]:
def get_basket_product_list_info(row: pd.Series):
    ruleset_info = row["RULESET"]
    ruleset_id = row["RULE_ID"]
    recomendation_info = row["RECOMENDATION"]

    def get_random_product(ls: list) -> str:
        if ls is not np.nan:
            return random.choice(ls)

    basket_info = [
                    get_random_product(row["MODEL_PROD_LIST_ITEM1"]),
                    get_random_product(row["MODEL_PROD_LIST_ITEM2"]),
                    get_random_product(row["MODEL_PROD_LIST_ITEM3"]),
                    get_random_product(row["MODEL_PROD_LIST_ITEM4"]),
                    get_random_product(row["MODEL_PROD_LIST_ITEM5"])
                   ]

    return ruleset_info, ruleset_id, recomendation_info, list(filter(None, basket_info))

In [ ]:
df_test = df.sample(n=30, random_state=2)
# df_test = df

In [ ]:
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36"
}
URL = "https://bpknsassat05.orlen.pl/api/getbasketoffer"   # URL = "http://10.1.111.45/api/getbasketoffer"
PLACEHOLDER = "####placeholder####"

In [ ]:
def clean_folder(folder_path: str) -> None:
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
clean_folder("data/request")
clean_folder("data/response")
clean_folder("data/report")

In [ ]:
def save_json_file(file_path: str, file_name: str, payload: str) -> bool:
    try:
        with open(f"{file_path}/{file_name}", "w", encoding="utf-8") as f:
            json.dump(payload, f, ensure_ascii=False, indent=2)
        return True
    except Exception as e:
        logging.error(f"Saving file error: {e}")
        return False

In [ ]:
def concate_values_in_series(series: pd.Series, cols: list) -> str:
    concatenation = ""
    for col in cols:
        if series[col] is not np.nan:
           concatenation = concatenation + ", " + series[col]
    return concatenation[2:]

In [ ]:
def dual_pos_api_test():
    # Pętla testująca metodę POST
    output_data = []
    l = len(df_test.index)
    j = 0
    for i, row in df_test.iterrows():
        j += 1
        print_progress_bar(j, l, prefix = 'Dual-POS API Testing Progress:', suffix = 'Complete', length = 100)
        ruleset_info, ruleset_id, recommendation_info, basket = get_basket_product_list_info(row)
        row_txt_data = replace_placeholder_in_text(template_file, PLACEHOLDER, basket)
        payload = json.loads(row_txt_data)

        cols = ["MODEL_PROD_CD_ITEM1", "MODEL_PROD_CD_ITEM2", "MODEL_PROD_CD_ITEM3", "MODEL_PROD_CD_ITEM4", "MODEL_PROD_CD_ITEM5"]
        basket_categoties = concate_values_in_series(row, cols)

        save_json_file("data/request", f"dual-pos-request-body_{i + 1}.json", payload)

        try:
            response = requests.post(URL, headers=HEADERS, json=payload, verify=False)
            response_json = response.json()
            save_json_file("data/response", f"dual-pos-response-body_{i + 1}.json", response_json)

            recomm_response = "Nie został zdefiniowany żaden treatment"
            try:
                recomm_response = response_json['outputs'][1]['value'][1]['data'][0]
            except TypeError as e:
                logging.info(recomm_response)

            response_row = {"RULESET": ruleset_info,
                            "RECOMMENDATION": recommendation_info,
                            "BASKET_CATEGORIES": basket_categoties,
                            "BASKET_PRODUCTS": basket,
                            "RESPONSE_CODE": response.status_code,
                            "RESPONSE": recomm_response}
            output_data.append(response_row)

        except requests.RequestException as e:
            logging.error(f"Request error: {e}")

        time.sleep(.05)

In [ ]:
dual_pos_api_test()

Dual-POS API Testing Progress: |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete


In [ ]:
output_df = pd.DataFrame(output_data)

output_df.sample(n=1)

,RULESET,RECOMMENDATION,BASKET_CATEGORIES,BASKET_PRODUCTS,RESPONSE_CODE,RESPONSE
10,ALL_UNNAMED_DUAL_POS,NAPOJE GAZOWANE,"NAPOJE NIEGAZ I HERB, WYROBY TYTONIOWE","[D05-001296, D06-010964]",200,Nie został zdefiniowany żaden treatment


In [ ]:
try:
    output_df.to_csv("data/report/dual-pos-test-report.csv", sep=";", header=True, index=False, encoding="utf-8-sig")
except Exception as e:
    logging.error(f"Saving file error: {e}")